In [ ]:
import pandas as pd 
import pymysql

In [ ]:
db_connection = pymysql.connect(
    read_default_file='/home/jupyter/.my.cnf',
    read_default_group='RAnalysis', 
    db = 'actin_personalization'
)

query = "SELECT * FROM knownPalliativeTreatments"

df = pd.read_sql(query, db_connection)

db_connection.close()

list(df.columns)

In [ ]:
from lifelines import CoxPHFitter
df_clean = df.dropna(subset = [
    'observedPfsDays',
    'hadProgressionEvent',
    'consolidatedTumorType',
    'ageAtDiagnosis',
    'whoStatusPreTreatmentStart',
    'tumorIncidenceYear'
])

def cox_model(df, formula):
    return CoxPHFitter().fit(
        df,
        duration_col="observedPfsDays",
        event_col="hadProgressionEvent",
        formula=formula,
        strata=["systemicTreatmentPlan"]
    )

full_model = cox_model(df_clean, "consolidatedTumorType + ageAtDiagnosis + whoStatusPreTreatmentStart + tumorIncidenceYear")
full_model.print_summary()

In [ ]:
reduced_model = cox_model(df_clean, "ageAtDiagnosis + whoStatusPreTreatmentStart + tumorIncidenceYear")

reduced_model.print_summary()

In [ ]:
def likelihood_ratio(llmin, llmax):
    return 2 * (llmax - llmin)

In [ ]:
from dataclasses import dataclass
from math import log10

@dataclass
class TestResult:
    feature: str
    survival_impact: float
    p_val: float
    dof_delta: int
    log_likelihood_ratio: float

# def is_complete(feature_result):
#     test_result = feature_result[1]
#     return len(test_result) > 3 and test_result[3] == 'OK' # and test_result[1] < 0.05

# complete_results = list(map(lambda kv: TestResult(kv[0], kv[1][2], kv[1][1], -log10(kv[1][1])), dict(filter(is_complete, result.items())).items()))

# result_df = pd.DataFrame(complete_results)

# result_df[result_df["p_val"] < 0.1]

In [ ]:
from scipy.stats.distributions import chi2
from sklearn.preprocessing import StandardScaler

base_df = df.dropna(subset = [
    'observedPfsDays',
    'hadProgressionEvent',
    'ageAtDiagnosis',
    'whoStatusPreTreatmentStart',
    'tumorIncidenceYear'
]).copy()


scaler = StandardScaler()
cols_to_standardize = [f for f in (['ageAtDiagnosis', 'whoStatusPreTreatmentStart', 'tumorIncidenceYear'] + features) if pd.api.types.is_numeric_dtype(df[f])]

base_df.loc[:, cols_to_standardize] = scaler.fit_transform(base_df.loc[:, cols_to_standardize])

def test_feature(df, simple_formula, feature):
    try:
        df_clean = df.dropna(subset = [feature])
        reduced_model = cox_model(df_clean, simple_formula)
        full_model = cox_model(df_clean, " + ".join([feature, simple_formula]))
        dof = 1 if pd.api.types.is_numeric_dtype(df[feature]) else df[feature].nunique() - 1
        log_likelihood_ratio = likelihood_ratio(reduced_model.log_likelihood_, full_model.log_likelihood_)
        p = chi2.sf(log_likelihood_ratio, dof)
        print(f"Testing feature {feature}: p-value {p}, exp(coeff) {full_model.hazard_ratios_[0]}")
        return TestResult(feature, full_model.hazard_ratios_[0], p, dof, log_likelihood_ratio)
    except Exception as e:
        print(f"Failed to test {feature}: {str(e)}")
        return None

# test_feature(base_df, "ageAtDiagnosis + whoStatusPreTreatmentStart + tumorIncidenceYear", "consolidatedTumorType")

In [ ]:
features = [
     'sex',
     'consolidatedTumorType',
     'hasHadPriorTumor',
     'cci',
     'cciNumberOfCategories',
     'cciHasAids',
     'cciHasCongestiveHeartFailure',
     'cciHasCollagenosis',
     'cciHasCopd',
     'cciHasCerebrovascularDisease',
     'cciHasDementia',
     'cciHasDiabetesMellitus',
     'cciHasDiabetesMellitusWithEndOrganDamage',
     'cciHasOtherMalignancy',
     'cciHasOtherMetastaticSolidTumor',
     'cciHasMyocardialInfarct',
     'cciHasMildLiverDisease',
     #'cciHasHemiplegiaOrParaplegia',
     'cciHasPeripheralVascularDisease',
     'cciHasRenalDisease',
     'cciHasLiverDisease',
     'cciHasUlcerDisease',
     'presentedWithIleus',
     'presentedWithPerforation',
     'anorectalVergeDistanceCategory',
     'hasMsi',
     'hasBrafMutation',
     'hasBrafV600EMutation',
     'hasRasMutation',
     'hasKrasG12CMutation',
     'asaClassificationPreSurgeryOrEndoscopy',
     'tumorBasisOfDiagnosis',
     'tumorLocation',
     'tumorDifferentiationGrade',
     'tnmCT',
     'tnmCN',
     'tnmCM',
     #'tnmPT',
     'tnmPN',
     'tnmPM',
     'stageCTNM',
     'stagePTNM',
     'stageTNM',
     'investigatedLymphNodesNumber',
     'positiveLymphNodesNumber',
     'distantMetastasesDetectionStatus',
     'numberOfLiverMetastases',
     'maximumSizeOfLiverMetastasisMm',
     'hasDoublePrimaryTumor',
     'mesorectalFasciaIsClear',
     'distanceToMesorectalFasciaMm',
     'venousInvasionDescription',
     'lymphaticInvasionCategory',
     'extraMuralInvasionCategory',
     'tumorRegression',
     'hasParticipatedInTrial',
     'metastasesSurgeries',
     'radiotherapies',
     #'metastasesRadiotherapies',
     #'response',
     'metastasisLocationGroupsPriorToSystemicTreatment'
]
base_formula = "ageAtDiagnosis + whoStatusPreTreatmentStart + tumorIncidenceYear"
result = [r for r in (test_feature(base_df, base_formula, f) for f in features) if r is not None]
result

In [ ]:
from math import log10

result_df = pd.DataFrame(result)
result_df["log10_p"] = result_df["p_val"].apply(lambda p: -log10(p))
result_df.head()

In [ ]:
import seaborn

seaborn.scatterplot(result_df[result_df["p_val"] < 0.05], x="survival_impact", y="log10_p")

In [ ]:
correlation = base_df[features].corr('pearson')

In [ ]:
seaborn.heatmap(correlation)

In [ ]:
anorectalVergeDistanceCategory_lookup = {
    "ZERO_TO_FIVE_CM": 2.5,
    "FIVE_TO_TEN_CM": 7.5,
    "TEN_TO_FIFTEEN_CM": 12.5,
    "OVER_FIFTEEN_CM": 17.5,
}
cciNumberOfCategories_lookup = {
    "ZERO_CATEGORIES": 0,
    "ONE_CATEGORY": 1,
    "TWO_OR_MORE_CATEGORIES": 2,
}

numberOfLiverMetastases_lookup = {
    "ONE": 1,
    "TWO": 2,
    "THREE": 3,
    "FOUR": 4,
    "FIVE_OR_MORE": 5,
    "MULTIPLE_BUT_EXACT_NUMBER_UNKNOWN": 3
}

asaClassificationPreSurgeryOrEndoscopy
venousInvasionDescription
lymphaticInvasionCategory
extraMuralInvasionCategory
tumorRegression

base_df['anorectalVergeDistanceCategory'] = base_df['anorectalVergeDistanceCategory'].apply(anorectalVergeDistanceCategory_lookup.get)
base_df['cciNumberOfCategories'] = base_df['cciNumberOfCategories'].apply(cciNumberOfCategories_lookup.get)
base_df['numberOfLiverMetastases'] = base_df['numberOfLiverMetastases'].apply(numberOfLiverMetastases_lookup.get)

In [ ]:
base_df['asaClassificationPreSurgeryOrEndoscopy'].value_counts()